In [ ]:
### At this point the ./src/main.py file needs to be executed with the preprocessed data. In the original training, Azure ML Studio was used to train the model. If you use another cloud computing service, you will need to adapt the code to your needs.

from azure.ai.ml import MLClient
from azure.ai.ml import command, Input, Output
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace, Dataset, Datastore
import os
import pandas as pd
import numpy as np

credential = DefaultAzureCredential()

import sys 
sys.path.append("../../")

from config import subscription_id, resource_group, workspace_name

ml_client = MLClient( 
    credential=credential,
    subscription_id = subscription_id,
    resource_group_name = resource_group,
    workspace_name = workspace_name
)

workspace = Workspace(subscription_id, resource_group, workspace_name)
datastore = Datastore.get(workspace, "workspaceblobstore")

## 1. Set up job environment (only first time)

In [ ]:
dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [ ]:
%%writefile {dependencies_dir}/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy
  - pip
  - pandas>=1.1,<1.2
  - pip:
    - inference-schema[numpy-support]==1.3.0
    - mlflow>=1.26.1
    - azureml-mlflow==1.42.0
    - psutil>=5.8,<5.9
    - tqdm>=4.59,<4.60
    - ipykernel~=6.0 
    - neuralprophet==1.0.0rc5
    #- git+https://github.com/ourownstory/neural_prophet.git
    #- git=https://github.com/Azure/azhpc-images/blob/master/ubuntu/ubuntu-22.x/ubuntu-22.04-hpc/install.sh
    - pytorch-lightning>=1.4.0
    - memory_profiler

## 2. Set up input data

In [ ]:
### Input data

data_path= 'azureml://subscriptions/workspaceblobstore/paths/LocalUpload/00_load_country.csv'

short_path = 'LocalUpload' + data_path.split('LocalUpload')[1] 
dataset = Dataset.Tabular.from_delimited_files(path=(datastore, short_path))
df = dataset.to_pandas_dataframe()

df['humidity'] = df['humidity'] / 100
df['precipitation'] = (df['precipitation'] - df['precipitation'].min()) / (df['precipitation'].max() - df['precipitation'].min())
df['cloud'] = df['cloud'] / 100
df['wind'] = (df['wind'] - df['wind'].min()) / (df['wind'].max() - df['wind'].min())

## 3. Start training job

In [ ]:
### Create your base command job

job = command( 
    code="./src",
    command=""" python main.py \
            --data ${{inputs.data}} \
            --results ${{outputs.results}} \
            """,
    environment="training_env@latest",
    inputs=dict(
        data=Input(
            type="uri_file",
            path=data_path
        ),
    ),
    outputs=dict(
        results=Output(type="uri_folder", 
                     mode="rw_mount"),
    ),
    compute="Standard-NC24ads-A100-v4-10nodes",
    display_name=f"country_forecast_2y/1y-5lagreg_v2",
    experiment_name="eu",
    description="Training NeuralProphet"
)

## Submit the job
ml_client.create_or_update(job)